# Data cleansing and transformation techniques

Here are two examples of data transformation:

**Cleaning Dirty Data:** Converting data from a dirty format to a clean format, ensuring consistency and accuracy.

**PDF Table Parsing:** Extracting all tables from a PDF file and saving the results into an Excel file, organizing the data for further analysis.

These examples demonstrate how to handle complex data sources and transform them into usable formats efficiently and accurately.

## Import libraries

In [2]:
#installing libraries
!pip install camelot-py[cv] -q
!pip install tabula-py -q
!pip install 'PyPDF2<3.0' -q
!pip install tabulate -q
!pip install pdfplumber -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 520.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 564.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.8/222.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 792.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 12.3 MB/s eta 0:00:00


In [3]:
import pandas as pd
import pdfplumber
import re
import tabula
import numpy as np

## Example 1 - Dirty data transformation

In [4]:
# read the file and get dirty data
dirty_data = pd.read_excel('Ditry Data Sample.xlsx', sheet_name='Dirty 1')
dirty_data.head(2)

,Segment>>,Consumer,Unnamed: 2,Unnamed: 3,Unnamed: 4,Consumer Total,Corporate,Unnamed: 7,Unnamed: 8,Unnamed: 9,Corporate Total,Home Office,Unnamed: 12,Unnamed: 13,Unnamed: 14,Home Office Total
0,Ship Mode>>,First Class,Same Day,Second Class,Standard Class,NaN,First Class,Same Day,Second Class,Standard Class,NaN,First Class,Same Day,Second Class,Standard Class,NaN
1,Order ID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#how clean data looks like
clean_data = pd.read_excel('Ditry Data Sample.xlsx', sheet_name='Clean 1')
clean_data.head(3)

,Segment,Ship Mode,OrderID,Sales
0,Consumer,First Class,CA-2011-103366,149.950
1,Consumer,First Class,CA-2011-109043,243.600
2,Consumer,First Class,CA-2011-113166,9.568


Let's start transforming the data to reach the result as clean_data.
* I plan to start from rebuilding the column names and assign Order ID as index.
* After that, I plan to melt the initial df, cause now the df looks more like pivot table, which need to be 'flatened'.
* Finally, I plan to lead the column names to the final look and compare the final df and clean_data to make sure I haven't missed anything.

In [6]:
# load the excel file with multi-level headers
file_path = 'Ditry Data Sample.xlsx'  # replace with your file name
sheet_name = 'Dirty 1'  # replace with your sheet name

df = pd.read_excel(file_path, sheet_name=sheet_name, header=[0, 1])

# flatten multi-level column headers
df.columns = ['_'.join(col).strip() for col in df.columns.values]

# set new index
df.set_index('Segment>>_Ship Mode>>', inplace=True)

# rename index
df.index.names = ['OrderID']

# drop the first row
df = df.iloc[1:]

# remove columns containing 'Total'
df = df.loc[:, ~df.columns.str.contains('Total')]

# melt df
melted_df = df.reset_index().melt(id_vars=['OrderID'], var_name='Combined', value_name='Sales')
melted_df.head(3)

,OrderID,Combined,Sales
0,CA-2011-100293,Consumer_First Class,NaN
1,CA-2011-100706,Consumer_First Class,NaN
2,CA-2011-100895,Consumer_First Class,NaN


So far, I've got melted df and now can move to final improvements and comparison to lean data.

In [7]:
#remove NaN values in Sales
df_with_fee = melted_df[~melted_df.Sales.isnull()]

#create a copy and split column names I concatenated before to get 2 separate columns
df_with_fee = df_with_fee.copy(deep = True)
df_with_fee[['Segment', 'Ship Mode']] = df_with_fee['Combined'].str.split('_', expand=True)

In [8]:
#select only columns that present in clean df
df_final = df_with_fee[['Segment', 'Ship Mode', 'OrderID', 'Sales']]
df_final.head(5)

,Segment,Ship Mode,OrderID,Sales
11,Consumer,First Class,CA-2011-103366,149.950
20,Consumer,First Class,CA-2011-109043,243.600
29,Consumer,First Class,CA-2011-113166,9.568
51,Consumer,First Class,CA-2011-124023,8.960
63,Consumer,First Class,CA-2011-130155,34.200


Now we can see that df_final looks pretty the same as clean_data. Let's see if the number of rows is the same.

In [9]:
print(df_final.shape)
print(clean_data.shape)

(834, 4)
(830, 4)


We can see that number of rows is not the same, so we need to heck, what rows are present in df_final and not present in clean_data.

In [10]:
# merge DataFrames on a common column to compare
comparison_df = df_final.merge(clean_data, on='OrderID', suffixes=('_df1', '_df2'), how='outer', indicator=True)

# filter rows where the values are not the same or rows present only in one DataFrame
different_rows = comparison_df[comparison_df['_merge'] != 'both']

In [11]:
different_rows

,Segment_df1,Ship Mode_df1,OrderID,Sales_df1,Segment_df2,Ship Mode_df2,Sales_df2,_merge
9,Consumer,First Class,CA-2011-164749,9.912,NaN,NaN,NaN,left_only
234,Consumer,Same Day,CA-2014-103065,59.824,NaN,NaN,NaN,left_only
235,Consumer,Same Day,CA-2014-122987,80.564,NaN,NaN,NaN,left_only
236,Consumer,Same Day,CA-2014-144498,817.829,NaN,NaN,NaN,left_only


In [12]:
#check if clean_data contains order id from different_rows
clean_data[clean_data.OrderID.isin(different_rows.OrderID.unique())]

,Segment,Ship Mode,OrderID,Sales


We can see that clean_data contains not all Order IDs from the dirty data. To reach absolute similarity, we should delete them. However, in practice, it's a better option to consult with client, whether he or she wants additional data to be saved or not.

In [13]:
df_final_remove = df_final[~df_final.OrderID.isin(different_rows.OrderID.unique())]
df_final_remove.reset_index(drop=True, inplace=True)

# check if the two DataFrames are identical
are_identical = df_final_remove.equals(clean_data)

print(f"Are the DataFrames identical? {are_identical}")

Are the DataFrames identical? True


## Example 2 - extrating info from pdf file

### Table 1

In [14]:
pdf_path = 'us_data.pdf'
offset = 9

# Open PDF and process each page
with pdfplumber.open(pdf_path) as pdf:
    texts = [page.extract_text() for page in pdf.pages]
    page_numbers = [re.findall(r"Table \d+.*?(\d+)$", text, re.MULTILINE) for text in texts if text]
    page_numbers = [item for sublist in page_numbers for item in sublist]  # Flatten list of lists

# Adjust page numbers considering the offset
actual_pages = [int(num) + offset - 1 for num in page_numbers]

# Process selected texts and extract data
for i in actual_pages:
    potential_table_lines = texts[i].split('\n') if texts[i] else []
    try:
        index = potential_table_lines.index('a Many of the published tables in the ACS5 are broken out by tenure.')
        data = potential_table_lines[:index][5:]
        print(data)
    except (ValueError, IndexError):
        continue


['Tenure Owner-Occupied and Renter-Occupied Included belowa TEN', 'Building year of 2010 and later, 2000-2009, 1980-1999, 1960-1979, B25036, YBL', 'first construction 1940-1959, and 1939 and before B25127b', 'Number of units in 1 Unit Detached, 1 Unit Attached, 2 Units, 3-4 Units, B25032, BLD', 'the building 5-9 Units, 10-19 Units, 20-49 Units, 50 and More B25124b,', 'Units, and Mobile and Other Units B25127b', 'Primary heating Utility Gas, Bottled Gas, Electricity, Fuel Oil, Wood, B25117 HFL', 'fuel type Coal, Solar, Other, and None', 'Number of 1-Person, 2-Person, 3-Person, 4-Person, 5-Person, B25009, NP', 'persons 6-Person, 7 or More Persons B25124b', 'Household income 0-5K, 5-10K, 10-15K, 15-20K, 20-25K, 25-35K, 35K- B25118 HINCP', '50K, 50-75K, 75-100K, 100-150K, 150K and more', 'Area median 0-30%, 30-50%, 50-80%, 80-100%, greater than not available not available', 'income 100%']


In [15]:
data

['Tenure Owner-Occupied and Renter-Occupied Included belowa TEN',
 'Building year of 2010 and later, 2000-2009, 1980-1999, 1960-1979, B25036, YBL',
 'first construction 1940-1959, and 1939 and before B25127b',
 'Number of units in 1 Unit Detached, 1 Unit Attached, 2 Units, 3-4 Units, B25032, BLD',
 'the building 5-9 Units, 10-19 Units, 20-49 Units, 50 and More B25124b,',
 'Units, and Mobile and Other Units B25127b',
 'Primary heating Utility Gas, Bottled Gas, Electricity, Fuel Oil, Wood, B25117 HFL',
 'fuel type Coal, Solar, Other, and None',
 'Number of 1-Person, 2-Person, 3-Person, 4-Person, 5-Person, B25009, NP',
 'persons 6-Person, 7 or More Persons B25124b',
 'Household income 0-5K, 5-10K, 10-15K, 15-20K, 20-25K, 25-35K, 35K- B25118 HINCP',
 '50K, 50-75K, 75-100K, 100-150K, 150K and more',
 'Area median 0-30%, 30-50%, 50-80%, 80-100%, greater than not available not available',
 'income 100%']

In [16]:
# Initialize DataFrame and define variables
df = pd.DataFrame(columns=['Variable', 'Categories', 'ACS5 Published Table', 'ACS5 Microdata Sample'])
variables = [
    'Tenure', 'Building year of first construction', 'Number of units in the building',
    'Primary heating fuel type', 'Number of persons', 'Household income', 'Area median income']

# Process data to handle line continuations and special cases
results = []
temp = data[0]
for item in data[1:]:
    if item[0].islower() or item[0].isdigit():
        temp += " " + item
    else:
        results.append(temp)
        temp = item
results.append(temp)  # Ensure the last item is appended

# Adjust specific index if needed before deletion to prevent index errors
if len(results) > 3:
    results[2] += " " + results.pop(3)

# Create dictionary and clean data
cleaned_data = {var: re.sub(r'\s+', ' ', re.sub(r'\b' + re.escape(var) + r'\b', '', res, flags=re.IGNORECASE).strip().strip(','))
                for var, res in zip(variables, results)}

# Fill DataFrame
for var, content in cleaned_data.items():
    print(f"{var}: {content}")


Tenure: Owner-Occupied and Renter-Occupied Included belowa TEN
Building year of first construction: Building year of 2010 and later, 2000-2009, 1980-1999, 1960-1979, B25036, YBL first construction 1940-1959, and 1939 and before B25127b
Number of units in the building: Number of units in 1 Unit Detached, 1 Unit Attached, 2 Units, 3-4 Units, B25032, BLD the building 5-9 Units, 10-19 Units, 20-49 Units, 50 and More B25124b, Units, and Mobile and Other Units B25127b
Primary heating fuel type: Primary heating Utility Gas, Bottled Gas, Electricity, Fuel Oil, Wood, B25117 HFL fuel type Coal, Solar, Other, and None
Number of persons: Number of 1-Person, 2-Person, 3-Person, 4-Person, 5-Person, B25009, NP persons 6-Person, 7 or More Persons B25124b
Household income: 0-5K, 5-10K, 10-15K, 15-20K, 20-25K, 25-35K, 35K- B25118 HINCP 50K, 50-75K, 75-100K, 100-150K, 150K and more
Area median income: Area median 0-30%, 30-50%, 50-80%, 80-100%, greater than not available not available income 100%


In [17]:
# Define regex patterns for later use
pattern_b2 = r'B2\d{4}[a-z]?'
pattern_microdata = r'\b(HINCP|[A-Z]{2,3})\b'
pattern_included_belowa = r'Included belowa'
pattern_not_available = r'not available'

# Iterate through each entry in cleaned_data
for variable, value in cleaned_data.items():
    # Use a set difference method to remove variable words from value
    variable_words = set(variable.split())
    value_words = set(value.split())
    remaining_words = value_words - variable_words
    value = ' '.join(word for word in value.split() if word in remaining_words)

    categories = []
    published_table = []
    microdata_sample = []

    if pattern_included_belowa in value:
        published_table.append(pattern_included_belowa)
        value = value.replace(pattern_included_belowa, '')

    if pattern_not_available in value:
        published_table.append(pattern_not_available)
        microdata_sample.append(pattern_not_available)
        value = value.replace(pattern_not_available, '')

    # Handle B2 codes
    b2_matches = re.findall(pattern_b2, value)
    if b2_matches:
        published_table.extend(b2_matches)
        value = re.sub(pattern_b2, '\n', value)  # Insert newline after B2 code

    # Handle microdata sample codes
    microdata_matches = re.findall(pattern_microdata, value)
    if microdata_matches:
        microdata_sample.extend(microdata_matches)
        value = re.sub(pattern_microdata, '', value)

    categories.append(value.strip())

    # Join categories into a single string, cleaning up excess newlines
    category_string = ' '.join(categories)
    category_string = re.sub(r'\n+', '\n', category_string)

    # Adjust newlines and commas
    category_string = re.sub(r',\n', '\n', category_string)  # Remove comma before newline

    # Create a temporary DataFrame to hold the processed data
    temp_df = pd.DataFrame({
        'Variable': [variable],
        'Categories': [category_string],
        'ACS5 Published Table': ['\n'.join(published_table) if all(re.match(pattern_b2, code) for code in published_table) else ', '.join(published_table)],
        'ACS5 Microdata Sample': [', '.join(microdata_sample)]
    })
    df = pd.concat([df, temp_df], ignore_index=True)
    df['Categories'] = df['Categories'].apply(lambda x: re.sub(r'\n[,\s]+', '\n', x).replace("  100%", "100%").strip())


df

,Variable,Categories,ACS5 Published Table,ACS5 Microdata Sample
0,Tenure,Owner-Occupied and Renter-Occupied,Included belowa,TEN
1,Building year of first construction,"2010 and later, 2000-2009, 1980-1999, 1960-197...",B25036\nB25127b,YBL
2,Number of units in the building,"1 Unit Detached, 1 Unit Attached, 2 Units, 3-4...",B25032\nB25124b\nB25127b,BLD
3,Primary heating fuel type,"Utility Gas, Bottled Gas, Electricity, Fuel Oi...",B25117,HFL
4,Number of persons,"1-Person, 2-Person, 3-Person, 4-Person, 5-Pers...",B25009\nB25124b,NP
5,Household income,"0-5K, 5-10K, 10-15K, 15-20K, 20-25K, 25-35K, 3...",B25118,HINCP
6,Area median income,"0-30%, 30-50%, 50-80%, 80-100%, greater than 100%",not available,not available


In [18]:
table1 = df

### Table 2

In [19]:
# Specify the path to your PDF file
file_path = 'us_data.pdf'

# Read tables from the PDF
tables = tabula.read_pdf(file_path, pages='all', multiple_tables=True)
table2 = tables[0]

# Insert the current column names as the first row and rename columns
table2.loc[-1] = table2.columns
table2.index = table2.index + 1
table2 = table2.sort_index().reset_index(drop=True)
table2.columns = ['Form', 'Description']

# Insert a row with NaNs between rows 3 and 4
nan_row = pd.DataFrame([[np.nan, np.nan]], columns=table2.columns)
table2 = pd.concat([table2.iloc[:3], nan_row, table2.iloc[3:]]).reset_index(drop=True)

def concat_columns_by_nan(df, column):
    new_column, temp_str, index_dict = [], '', {}
    start_index = 0

    for index, value in enumerate(df[column]):
        if pd.isna(value) and temp_str:
            new_column.extend([temp_str.strip()] * (index - len(new_column)))
            index_dict[temp_str.strip()] = list(range(start_index, index))
            temp_str = ''
        elif not pd.isna(value):
            if temp_str == '':
                start_index = index
            temp_str += str(value) + '\n'

    if temp_str:
        new_column.extend([temp_str.strip()] * (len(df) - len(new_column)))
        index_dict[temp_str.strip()] = list(range(start_index, len(df)))

    df[column] = new_column
    return df, index_dict

# Process the DataFrame
df_new, form_indices = concat_columns_by_nan(table2, 'Form')

# Prepare the final DataFrame
description_values = table2.Description.unique()
final_form, final_description = [], []

for form, indices in form_indices.items():
    final_form.append(form)
    concatenated_description = "\n".join([description_values[i] for i in indices if i < len(description_values) and not pd.isna(description_values[i])])
    final_description.append(concatenated_description)

final_df = pd.DataFrame({
    'Form': final_form,
    'Description': final_description
})

# Print the final DataFrame
final_df


,Form,Description
0,EIA Form 861,Form 861 is a survey that gathers data on elec...
1,EIA Form 176,Form 176 provides natural gas sales data at th...
2,EIA CBECS,CBECS is a survey that collects sample informa...
3,Federal Emergency\nManagement\nAdministration’...,The Federal Emergency Management Administratio...
4,"CoStar Realty\nInformation, Inc.\n(CoStar)",CoStar provides commercial building data for p...
5,ABB/Ventyx\nEnergy Velocity\nSuite Utility\nTe...,The proprietary ABB Velocity Suite platform pr...


In [20]:
table2 = final_df

### Table 3

In [21]:
table3 = tables[1]
table3

,Unnamed: 0,Building,Building.1,Building.2,Building.3,Total MWh,Total MWh.1
0,NaN,Counts,Counts,Square,Square,Consumption,Consumption
1,NaN,(CoStar),(GBS),Footage,Footage,(CoStar),(GBS)
2,NaN,NaN,NaN,(CoStar),(GBS),NaN,NaN
3,Rural,"7,314","22,190","141,592,708","143,718,864","2,759,376","3,941,318"
4,Urban,"45,430","82,861","916,964,757","545,993,028","17,379,394","16,272,724"


In [22]:
# Create new column names by concatenating the first few rows and the column headers
new_columns = []
for col in table3.columns:
    new_col_name = '\n'.join([str(x) for x in table3[col][:3] if pd.notna(x)])
    new_columns.append(new_col_name)

# Assign new column names and drop the first three rows
table3.columns = new_columns
table3 = table3.iloc[3:].reset_index(drop=True)

table3

,,Counts\n(CoStar),Counts\n(GBS),Square\nFootage\n(CoStar),Square\nFootage\n(GBS),Consumption\n(CoStar),Consumption\n(GBS)
0,Rural,"7,314","22,190","141,592,708","143,718,864","2,759,376","3,941,318"
1,Urban,"45,430","82,861","916,964,757","545,993,028","17,379,394","16,272,724"


### Table 4

In [23]:
pdf_path = 'us_data.pdf'
offset = 9

# Open PDF and process each page
with pdfplumber.open(pdf_path) as pdf:
    texts = [page.extract_text() for page in pdf.pages]
    page_numbers = [re.findall(r"Table \d+.*?(\d+)$", text, re.MULTILINE) for text in texts if text]
    page_numbers = [item for sublist in page_numbers for item in sublist]  # Flatten list of lists

# Adjust page numbers considering the offset
actual_pages = [int(num) + offset - 1 for num in page_numbers]

# Process selected texts and extract data
for i in actual_pages:
    potential_table_lines = texts[i].split('\n') if texts[i] else []
    try:
        index = potential_table_lines.index('23 Jurisdictional boundaries are based on the U.S. Census Bureau 2013 Topologically Integrated Geographic')
        index2 = potential_table_lines.index('Table 4. Example of Mapping between Cities and Tracts with Weights')
        data = potential_table_lines[:index][index2+1:]
        print(data)
    except (ValueError, IndexError):
        continue


['Tract ID City ID Weight', '100001 01 1', '100002 01 0.5']
['Tract ID City ID Weight', '100001 01 1', '100002 01 0.5']


In [24]:
data

['Tract ID City ID Weight', '100001 01 1', '100002 01 0.5']

In [25]:
# Split the first row into columns
columns = data[0].split()

# Adjust for the separated 'ID'
adjusted_columns = []
i = 0
while i < len(columns):
    if i+1 < len(columns) and columns[i+1] == "ID":
        adjusted_columns.append(f"{columns[i]} {columns[i+1]}")
        i += 2  # Skip the next column because it's part of the current one
    else:
        adjusted_columns.append(columns[i])
        i += 1

# Split the remaining rows into data
data_rows = [row.split() for row in data[1:]]

# Create the DataFrame
df = pd.DataFrame(data_rows, columns=adjusted_columns)

# Display the DataFrame
df

,Tract ID,City ID,Weight
0,100001,01,1
1,100002,01,0.5


In [26]:
table4 = df

### Table 5

In [27]:
table5 = tables[2]

# Insert the current column names as the first row and rename columns
table5.loc[-1] = table5.columns
table5.index = table5.index + 1
table5 = table5.sort_index().reset_index(drop=True)
table5.columns = ['Form', 'Description']
table5

,Form,Description
0,EIA Form 861,Form 861 is a survey that gathers data on elec...
1,EIA Form 176,Form 176 provides natural gas sales data at th...
2,Industrial Energy\rTool (IET) county-\rlevel e...,"NREL developed the IET to be “an open-source, ..."
3,Homeland Security\rInfrastructure\rFoundation-...,The HIFLD program was established by the HIFLD...
4,EPA Greenhouse\rGas Reporting\rProgram (GHGRP),The GHGRP tracks the GHG emissions and energy ...


### Save tables to Excel file

In [28]:
dfs = [table1, table2, table3, table4, table5]
sheet_names = ['Table1', 'Table2', 'Table3', 'Table4', 'Table5']

# Write each DataFrame to a separate sheet in an Excel file
with pd.ExcelWriter('tables.xlsx') as writer:
    for df, sheet in zip(dfs, sheet_names):
        df.to_excel(writer, sheet_name=sheet, index=False)